In [18]:
import time
import boto3
import os
import configparser

In [3]:
!ls ../data

29_2150_compressed_GlobalLandTemperaturesByCity.csv.zip
29_2150_compressed_GlobalLandTemperaturesByCountry.csv.zip
29_2150_compressed_GlobalLandTemperaturesByMajorCity.csv.zip
29_2150_compressed_GlobalLandTemperaturesByState.csv.zip
I94_SAS_Labels_Descriptions.SAS
airport-codes_csv.csv
capstone-user.csv
datasets_29_2150_GlobalTemperatures.csv
immigration_data_sample.csv
output
sas_data
sub-est2019_all.csv
us-cities-demographics.csv
us_states.csv


In [6]:
config_aws = configparser.ConfigParser()
config_aws.read_file(open('../aws_credentials.cfg'))

KEY                    = config_aws.get('AWS','KEY')
SECRET                 = config_aws.get('AWS','SECRET')
ARN                    = config_aws.get('AWS', 'IAM_ARN')

In [24]:
config_s3 = configparser.ConfigParser()
config_s3.read_file(open('../aws_setup.cfg'))

S3_REGION              = config_s3.get('S3', 'REGION')
S3_BUCKET_NAME         = config_s3.get('S3', 'NAME')

In [45]:
s3 = boto3.resource(
    's3',
    aws_access_key_id=KEY,
    aws_secret_access_key= SECRET
)

In [8]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=KEY,
    aws_secret_access_key= SECRET
)

In [27]:
response = s3_client.create_bucket(
    ACL='public-read-write',
    Bucket=S3_BUCKET,
    CreateBucketConfiguration={
        'LocationConstraint': S3_REGION
    },
)

In [30]:
print(response)

{'ResponseMetadata': {'RequestId': '240C11DBCA15E2B4', 'HostId': '2Pp1a76WSSQIC0rTqaijJa2UvWOQwUKlytXvyMrsTIyEPsLEPY9eEYlgvF8DDbi/xrmqnpLZWR0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2Pp1a76WSSQIC0rTqaijJa2UvWOQwUKlytXvyMrsTIyEPsLEPY9eEYlgvF8DDbi/xrmqnpLZWR0=', 'x-amz-request-id': '240C11DBCA15E2B4', 'date': 'Sat, 12 Sep 2020 16:10:58 GMT', 'location': 'http://immigrate-demographics-s3-1629.s3.amazonaws.com/', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Location': 'http://immigrate-demographics-s3-1629.s3.amazonaws.com/'}


In [29]:
def upload_folder(s3_client, folder_path, file_key):
    file_path = os.path.join(folder_path, file_key)
    for root, dirs, files in os.walk(file_path):
        for file in files:
            upload_from = os.path.join(root, file)
            upload_to = os.path.join(file_key, file)
            s3_client.upload_file(upload_from, S3_BUCKET_NAME, upload_to)

In [30]:
st = time.time()
upload_folder(s3_client, '..', 'data/sas_data')
print(f"===Uploaded files to S3. Used time {(time.time() - st)/60:5.2f}min")

../data/sas_data part-00003-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/sas_data .part-00003-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc
../data/sas_data .part-00012-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc
../data/sas_data part-00008-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/sas_data .part-00011-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc
../data/sas_data part-00004-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/sas_data part-00013-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/sas_data .part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc
../data/sas_data part-00005-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/sas_data ._SUCCESS.crc
../data/sas_data .part-00008-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc
../data/sas_data part-00012-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet
../data/s

In [31]:
def upload_file(s3_client, folder_path, file_key):
    upload_from = os.path.join(folder_path, file_key)
    upload_to = os.path.join(file_key)
    s3_client.upload_file(upload_from, S3_BUCKET_NAME, upload_to)

In [32]:
upload_file(s3_client, '..', 'data/immigration_data_sample.csv')

In [33]:
upload_file(s3_client, '..', 'data/us_states.csv')

In [34]:
upload_file(s3_client, '..', 'data/I94_SAS_Labels_Descriptions.SAS')

In [35]:
upload_file(s3_client, '..', 'data/us-cities-demographics.csv')

In [36]:
def create_folder(s3_client, folder_name):
    s3_client.put_object(ACL='public-read-write',
                     Bucket=S3_BUCKET_NAME, 
                     Key=folder_name+'/')
    

In [37]:
create_folder(s3_client, 'logs')

In [49]:
s3.Bucket(S3_BUCKET_NAME) in s3.buckets.all()

True

In [43]:
s3_client.get_object(Bucket=S3_BUCKET_NAME)

ParamValidationError: Parameter validation failed:
Missing required parameter in input: "Key"